# Visualizations
This notebook contains some code snippets to generate plots from CSV files for 3DGS results from TensorBoard etc.

In [ ]:
import csv
from pathlib import Path

from PIL import Image
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300


## Image stiching

In [ ]:
# Code mainly generated from github copilot, boilerplate for generating nice plots
def stitch_images_in_columns(image_tuples):
    """
    Stitch a series of PIL images into a single image, arranging them in columns.

    This function takes a list of tuples, where each tuple contains PIL Image objects.

    Parameters:
    - image_tuples (list of tuples): A list where each tuple contains PIL Image objects.
      All tuples must have the same length. The images in position i of each tuple are
      stitched together in column i. For example, [(img1, img2), (img3, img4)] will
      stitch img1 and img3 in the first column, and img2 and img4 in the second column.

    Returns:
    - PIL.Image: A new PIL Image object containing the stitched images.

    Example:
    >>> from PIL import Image
    >>> img1 = Image.open('path/to/img1.jpg')
    >>> img2 = Image.open('path/to/img2.jpg')
    >>> img3 = Image.open('path/to/img3.jpg')
    >>> img4 = Image.open('path/to/img4.jpg')
    >>> combined_image = stitch_images_in_columns([(img1, img2), (img3, img4)])
    >>> combined_image.show()
    """
    # Determine the total width and maximum height for the new image
    total_width = 0
    max_height_per_column = [0] * len(image_tuples[0])  # Assuming all tuples have the same length

    # Calculate total width and max height per column
    for images in image_tuples:
        for i, img in enumerate(images):
            total_width += img.width if i == 0 else 0  # Add width only for the first column
            max_height_per_column[i] = max(max_height_per_column[i], img.height)

    total_height = sum(max_height_per_column)  # Total height is the sum of max heights of each column

    # Create a new image with the calculated dimensions
    new_img = Image.new('RGB', (total_width, total_height))

    # Paste images into the new image
    y_offset = 0
    for column_index in range(len(image_tuples[0])):
        x_offset = 0  # Reset x_offset for each column

        for images in image_tuples:
            new_img.paste(images[column_index], (x_offset, y_offset))
            x_offset += images[column_index].width  # Increment x_offset by the width of the current image

        y_offset += max_height_per_column[column_index]  # Increment y_offset by the height of the current column

    return new_img

def save_and_caption(image_tuples, captions, output_path):
    # First, stitch the images in columns using the previously defined function
    combined_image = stitch_images_in_columns(image_tuples)

    # Convert the PIL image to a NumPy array for matplotlib
    combined_image_np = np.array(combined_image)

    # Create a matplotlib figure and axis
    fig, ax = plt.subplots(figsize=(10, 8), dpi=900)
    ax.imshow(combined_image_np)
    ax.axis('off')  # Hide the axis

    # Add captions for each column
    total_height = combined_image.height
    num_rows = len(image_tuples[0])
    row_height = total_height / num_rows
    fontdict = {'fontname': 'Times New Roman'}

    for i, caption in enumerate(captions):
        ax.text(-200,i * row_height + row_height / 2, caption, ha='center', va='center', fontdict=fontdict)

    # Save the figure
    # plt.savefig(output_path, bbox_inches='tight')
    plt.show()
    # plt.close(fig)  # Close the figure to free memory

In [ ]:
image_dir = Path("/usr/stud/kaa/thesis/DEN-Splatting/evaluation/gaussian_splatting_pics/ds01")

frame260_paths = (
image_dir / "quality_260_gt.png",
# image_dir / "quality_260_colmap.png",
image_dir / "quality_260_colmap_masked.png",
image_dir / "quality_260_colmap_dense_masked.png",
image_dir / "quality_260_colmap_dense_masked_with_init.png",
image_dir / "quality_260_colmap_dense_masked_with_skydome.png",
image_dir / "quality_260_colmap_dense_masked_with_init_and_skydome.png",
)

frame340_paths = (
image_dir / "quality_340_gt.png",
image_dir / "quality_340_colmap_masked.png",
image_dir / "quality_340_colmap_dense_masked.png",
image_dir / "quality_340_colmap_dense_masked_with_init.png",
image_dir / "quality_340_colmap_dense_masked_with_skydome.png",
image_dir / "quality_340_colmap_dense_masked_with_init_and_skydome.png",
)

frame600_paths = (
image_dir / "quality_600_gt.png",
image_dir / "quality_600_colmap_masked.png",
image_dir / "quality_600_colmap_dense_masked.png",
image_dir / "quality_600_colmap_dense_masked_with_init.png",
image_dir / "quality_600_colmap_dense_masked_with_skydome.png",
image_dir / "quality_600_colmap_dense_masked_with_init_and_skydome.png",
)

frame260_images = tuple([Image.open(p) for p in frame260_paths])
frame340_images = tuple([Image.open(p) for p in frame340_paths])
frame600_images = tuple([Image.open(p) for p in frame600_paths])

images = [frame260_images, frame340_images, frame600_images]
captions=["GT", "COLMAP", "Dense",  "Dense + Init", "Dense\n+ Skydome", "Dense + Init\n + Skydome"]
save_and_caption(images, captions, "")


## 3DGS plots from csv
Useful for downloading csv data from tensorboard and getting matplotlib plots from'em

In [ ]:
def parse_csvs(paths, labels):
    results = {}
    for p, label in zip(paths, labels):
        with open(p, newline="") as csvfile:
            reader = csv.reader(csvfile, delimiter=",")
            next(reader, None) # skips the header
            x = []
            y = []
            for row in reader:
                _, iteration, value = row
                x.append(float(iteration))
                y.append(float(value))
            results[label] = (x, y)
    return results

In [ ]:
paths_to_psnr = []
paths_to_l1 = []
paths_to_ssim = []
labels = []

# eval_dir = Path("/usr/stud/kaa/thesis/DEN-Splatting/evaluation/gaussian_splatting_plots")
# paths_to_psnr.append(eval_dir / "1_colmap_baseline_psnr.csv")
# paths_to_l1.append(eval_dir / "1_colmap_baseline_l1.csv")
# paths_to_ssim.append(eval_dir / "1_colmap_baseline_ssim.csv")
# labels.append("COLMAP Baseline (no mask)")

# paths_to_psnr.append(eval_dir / "2_colmap_dense_psnr.csv")
# paths_to_l1.append(eval_dir / "2_colmap_dense_l1.csv")
# paths_to_ssim.append(eval_dir / "2_colmap_dense_ssim.csv")
# labels.append("COLMAP Dense No Masking")

eval_dir = Path("/usr/stud/kaa/thesis/DEN-Splatting/evaluation/gaussian_splatting_plots")
paths_to_psnr.append(eval_dir / "6_colmap_baseline_masked_psnr.csv")
paths_to_l1.append(eval_dir / "6_colmap_baseline_masked_l1.csv")
paths_to_ssim.append(eval_dir / "6_colmap_baseline_masked_ssim.csv")
labels.append("COLMAP Baseline")

paths_to_psnr.append(eval_dir / "3_colmap_dense_masked_psnr.csv")
paths_to_l1.append(eval_dir / "3_colmap_dense_masked_l1.csv")
paths_to_ssim.append(eval_dir / "3_colmap_dense_masked_ssim.csv")
labels.append("COLMAP Dense")

paths_to_psnr.append(eval_dir / "4_colmap_dense_masked_with_init_psnr.csv")
paths_to_l1.append(eval_dir / "4_colmap_dense_masked_with_init_l1.csv")
paths_to_ssim.append(eval_dir / "4_colmap_dense_masked_with_init_ssim.csv")
labels.append("COLMAP Dense + Init")

paths_to_psnr.append(eval_dir / "11_colmap_dense_masked_with_skydome_psnr.csv")
paths_to_l1.append(eval_dir / "11_colmap_dense_masked_with_skydome_l1.csv")
paths_to_ssim.append(eval_dir / "11_colmap_dense_masked_with_skydome_ssim.csv")
labels.append("COLMAP Dense + Skydome")

paths_to_psnr.append(eval_dir / "5_colmap_dense_masked_with_init_and_skydome_psnr.csv")
paths_to_l1.append(eval_dir / "5_colmap_dense_masked_with_init_and_skydome_l1.csv")
paths_to_ssim.append(eval_dir / "5_colmap_dense_masked_with_init_and_skydome_ssim.csv")
labels.append("COLMAP Dense + Init + Skydome")

fig = plt.figure(figsize=(18,6))

# psnr
ax1 = fig.add_subplot(1,3,1)
results_psnr = parse_csvs(paths_to_psnr, labels)
for label, (x, y) in results_psnr.items():
    ax1.plot(x, y, label=label)
ax1.legend()
ax1.set_title("PSNR")
ax1.set_xlabel("Iterations")
ax1.set_ylabel("PSNR")
ax1.grid()

# ssim
ax1 = fig.add_subplot(1,3,2)
results_ssim = parse_csvs(paths_to_ssim, labels)
for label, (x, y) in results_ssim.items():
    ax1.plot(x, y, label=label)
ax1.set_title("SSIM")
ax1.set_xlabel("Iterations")
ax1.set_ylabel("SSIM")
ax1.grid()

# l1
ax3 = fig.add_subplot(1,3,3)
results_l1 = parse_csvs(paths_to_l1, labels)
for label, (x, y) in results_l1.items():
    ax3.plot(x, y, label=label)
# ax3.legend()
ax3.set_title("L1")
ax3.set_xlabel("Iterations")
ax3.set_ylabel("L1 Loss")
ax3.grid()

plt.show()

## Point cloud renderings from perturbed views using open3d
Useful for getting consistent images of dense point cloud reconstructions

In [ ]:
from pathlib import Path

import torch
import open3d as o3d

from modules.core.utils import format_intrinsics
from modules.io.utils import read_ply, read_ply_o3d, save_image_torch
from modules.io.datasets import ColmapDataset, KITTI360Dataset, CustomDataset
from modules.scale_alignment.sparse import project_pcd_o3d



root_dir = Path("/usr/stud/kaa/data/root/kitti360_0_mini")
ply_path = Path("/usr/stud/kaa/data/root/kitti360_0_mini/reconstructions/4_colmap_sparse_scale/cloud.ply")
pose_scale = 41.6
colmap_dir = root_dir / "poses" / "colmap"
image_dir = root_dir / "data" / "rgb"
output_dir = Path("./pcd_projections")
pose_ids = [500, 800]
seq_id = 0
cam_id = 0
# pose_path = Path("")
pose_path = None
depth_max = 30
target_size = ()
padded_img_name_length = 10
dataset_type = "colmap" # kitti360, colmap, or custom
intrinsics = [552.55, 552.55, 682.05, 238.77] # [fx, fy, cx, cy]

# pose rotation
angle = 15
radian = angle * torch.pi / 180
R = torch.tensor([[torch.cos(radian), -torch.sin(radian), 0.0],
                    [torch.sin(radian),  torch.cos(radian), 0.0],
                    [0.0,                0.0,               1.0]])
#TODO figure out what we have to add here to disturb the view



if dataset_type == "kitti360":
    dataset = KITTI360Dataset(seq_id, cam_id, pose_scale, target_size)
    pose_path = dataset.pose_path # GT poses
elif dataset_type == "colmap":
    dataset = ColmapDataset(
    colmap_dir,
    pose_scale=pose_scale,
    target_size=target_size,
    orig_intrinsics=intrinsics,
    padded_img_name_length=padded_img_name_length,
    )
    if pose_path is not None:
        dataset.pose_path = pose_path
elif dataset_type == "custom":
    dataset = CustomDataset(
        image_dir,
        pose_path,
        pose_scale=pose_scale,
        target_size=target_size,
        orig_intrinsics=intrinsics,
        padded_img_name_length=padded_img_name_length,
        )

H, W = dataset.H, dataset.W

if ply_path.name == "points3D.txt":
    pcd = ColmapDataset.read_colmap_pcd_o3d(ply_path, convert_to_float32=True)
else:
    pcd = read_ply_o3d(ply_path, convert_to_float32=True)

_, _, poses = dataset.get_by_frame_ids(pose_ids)
K = format_intrinsics(intrinsics) # [3, 3]

images = []
for pose in poses:
    pose_inv = torch.linalg.inv(pose)

    rgb = project_pcd_o3d(
        pcd,
        W,
        H,
        K,
        pose_inv,
        depth_max,
        get_rgb=True,
        )

    images.append(rgb)

for i, image in enumerate(images):
    save_image_torch(image, f"projection_{i}", output_dir=output_dir)

print("Done!")
